In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
import evaluate

In [5]:
# Load & encode dataset
df = pd.read_csv("cleaned_insurance_dataset.csv")
le = LabelEncoder()
df["label"] = le.fit_transform(df["intent"])  # convert text labels to integers

In [6]:
# Save mapping for inference
label2id = {label: idx for idx, label in enumerate(le.classes_)}
id2label = {idx: label for label, idx in label2id.items()}


In [7]:
# Split into train/test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [8]:
# Convert to HF datasets
train_dataset = Dataset.from_pandas(train_df[["text", "label"]])
test_dataset = Dataset.from_pandas(test_df[["text", "label"]])


In [9]:
# Tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

Map: 100%|██████████| 2845/2845 [00:00<00:00, 98303.73 examples/s]


In [10]:
# Model
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Metrics
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

In [12]:
def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=1)
    labels = pred.label_ids
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels, average="weighted")["f1"]
    }


In [13]:

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)


In [14]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/var/folders/l6/3q9r80gx2z1d5gsj9vgzz7tw0000gn/T/ipykernel_56558/2063252845.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
# Train
trainer.train()

/Users/mariebiskoot/Documents/VoiceAgent/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.108500,0.164128,0.988049,0.987739
2,0.186800,0.026030,0.996485,0.996446
3,0.021300,0.016771,0.998243,0.998216


/Users/mariebiskoot/Documents/VoiceAgent/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/mariebiskoot/Documents/VoiceAgent/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=2136, training_loss=0.32063933849781195, metrics={'train_runtime': 271.1596, 'train_samples_per_second': 125.904, 'train_steps_per_second': 7.877, 'total_flos': 238607086098216.0, 'train_loss': 0.32063933849781195, 'epoch': 3.0})

In [16]:
# Save model + label mapping
model.save_pretrained("insurance_intent_model")
tokenizer.save_pretrained("insurance_intent_model")
with open("label_map.txt", "w") as f:
    for k, v in label2id.items():
        f.write(f"{k}:{v}\n")